In [221]:
from pprint import pprint
from numpy import random

In [245]:
#initialization of the grid

#spaces are open for movement
# ■ represents walls
symbols = [' ', '■']
cols = 10
rows = 10

#we want to get from starting position S to ending position E
#in the fewest moves possible

#first we define a class representing a point on the grid
#it has position x and y
# value ' '
# and a value for the num of steps to get there
class Point:
    def __init__(self, x, y, value=' ', steps=0):
        self.x = x
        self.y = y
        self.value = value
        self.repr = ''.join([str(self.x), str(self.y)])
        self.steps = steps
    def __str__(self):
        return str({'x' : self.x, 'y':self.y, 'value':self.value})
    def __repr__(self):
        return ''.join([str(self.x), str(self.y)])

starting_point = Point(
    x = random.choice(range(cols)),
    y = random.choice(range(rows)),
    value='S')
ending_point = Point(
    x = random.choice(range(cols)),
    y = random.choice(range(rows)),
    value='E')

grid = []
for r in range(rows):
    row = []
    for c in range(cols):
        if r == starting_point.x and c == starting_point.y:
            row.append(starting_point.value)
        elif r == ending_point.x and c == ending_point.y:
            row.append(ending_point.value)
        else:
            row.append(random.choice(symbols, 1, p=[0.7, 0.3])[0])
    grid.append(row)

In [246]:
grid

[[' ', ' ', ' ', ' ', ' ', '■', '■', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', '■', ' ', ' ', ' ', ' ', ' '],
 ['■', ' ', 'S', '■', ' ', ' ', ' ', ' ', '■', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', '■', '■', '■', ' '],
 [' ', '■', ' ', ' ', '■', '■', ' ', '■', ' ', '■'],
 [' ', ' ', ' ', '■', ' ', '■', '■', ' ', ' ', ' '],
 ['■', '■', ' ', '■', ' ', ' ', ' ', ' ', ' ', '■'],
 ['■', ' ', ' ', '■', '■', ' ', ' ', '■', ' ', '■'],
 [' ', ' ', ' ', '■', '■', '■', '■', ' ', '■', '■'],
 ['■', 'E', '■', ' ', '■', ' ', ' ', ' ', ' ', ' ']]

In [247]:
#we define a function to get possible next movement
#in the form of neighbors that are not wall
def getNeighbors(point, grid, current_steps):
    queue = []
    num_rows = len(grid)
    num_cols = len(grid[0])
    x = point.x
    y = point.y
    steps=current_steps+1
    #up
    if point.x > 0:
        newpoint = Point(x=x-1,y=y,value=grid[x-1][y],steps=steps)
        queue.append(newpoint)
    #down
    if x < num_rows -1:
        newpoint = Point(x=x+1,y=y,value=grid[x+1][y],steps=steps)
        queue.append(newpoint)
    #left
    if y > 0:
        newpoint = Point(x=x,y=y-1,value=grid[x][y-1],steps=steps)
        queue.append(newpoint)
    #right
    if y < num_cols -1:
        newpoint = Point(x=x,y=y+1,value=grid[x][y+1],steps=steps)
        queue.append(newpoint)
    queue = [m for m in queue if m.value != '■']
    return queue

In [248]:
#bds

frontier = getNeighbors(starting_point, grid,0)
came_from = {}
#for i in range(rows):
#    for j in range(cols):
#        came_from[''.join([str(i), str(j)])] = []
        
for f in frontier:
    came_from[f.repr] = starting_point
came_from[starting_point.repr] = None

min_steps = 999999
while frontier:
    current = frontier.pop(0)
    steps = current.steps
    for n in getNeighbors(current, grid,steps):
        if n.value == 'E':
            if n.steps <= min_steps:
                came_from[n.repr] = current
                min_steps = n.steps
            pass
        elif n.repr not in came_from:
            frontier.append(n)
            came_from[n.repr] = current
            

In [261]:
#draw the path
current = came_from[ending_point.repr]
path = []
while current.repr != starting_point.repr:
    path.append(current.repr)
    current = came_from[current.repr]
for p in path:
    cell = [int(x) for x in list(p)]
    grid[cell[0]][cell[1]] = '*'

In [262]:
grid

[[' ', ' ', ' ', ' ', ' ', '■', '■', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', '■', ' ', ' ', ' ', ' ', ' '],
 ['■', ' ', 'S', '■', ' ', ' ', ' ', ' ', '■', ' '],
 [' ', ' ', '*', ' ', ' ', ' ', '■', '■', '■', ' '],
 [' ', '■', '*', ' ', '■', '■', ' ', '■', ' ', '■'],
 [' ', ' ', '*', '■', ' ', '■', '■', ' ', ' ', ' '],
 ['■', '■', '*', '■', ' ', ' ', ' ', ' ', ' ', '■'],
 ['■', ' ', '*', '■', '■', ' ', ' ', '■', ' ', '■'],
 [' ', '*', '*', '■', '■', '■', '■', ' ', '■', '■'],
 ['■', 'E', '■', ' ', '■', ' ', ' ', ' ', ' ', ' ']]

Dijkstra with movements costs

In [313]:
#initialization of the grid

#spaces are open for movement
# ■ represents walls (cost = infinite)
# ░ represents swamp (cost = 5)
symbols = [' ', '■', '░']
mov_cost = {' ' : 1, '░' : 5, '■' : 9999, 'S' : 9999, 'E' : 9999}
costs = {}
cols = 10
rows = 10

#we want to get from starting position S to ending position E
#in the fewest moves possible

#first we define a class representing a point on the grid
#it has position x and y
# value ' '
# and a value for the num of steps to get there
class Point:
    def __init__(self, x, y, value=' ', steps=0):
        self.x = x
        self.y = y
        self.value = value
        self.repr = ''.join([str(self.x), str(self.y)])
        self.steps = steps
    def __str__(self):
        return str({'x' : self.x, 'y':self.y, 'value':self.value})
    def __repr__(self):
        return ''.join([str(self.x), str(self.y)])

starting_point = Point(
    x = random.choice(range(cols)),
    y = random.choice(range(rows)),
    value='S')
ending_point = Point(
    x = random.choice(range(cols)),
    y = random.choice(range(rows)),
    value='E')

grid = []
for r in range(rows):
    row = []
    for c in range(cols):
        if r == starting_point.x and c == starting_point.y:
            row.append(starting_point.value)
            costs[''.join([str(r), str(c)])] = 9999
        elif r == ending_point.x and c == ending_point.y:
            row.append(ending_point.value)
            costs[''.join([str(r), str(c)])] = 9999
        else:
            to_append = random.choice(symbols, 1, p=[0.5, 0.2, 0.3])[0]
            costs[''.join([str(r), str(c)])] = mov_cost[to_append]
            row.append(to_append)
    grid.append(row)

In [314]:
grid

[['░', '░', ' ', '░', ' ', '░', '░', ' ', 'E', ' '],
 ['░', ' ', '■', '■', '░', '■', ' ', '░', ' ', ' '],
 [' ', '░', '░', '░', '■', ' ', ' ', '░', ' ', ' '],
 [' ', '░', ' ', ' ', '░', ' ', '■', ' ', '■', '░'],
 [' ', '■', '■', '░', ' ', ' ', '░', '░', '░', '■'],
 ['■', ' ', '░', ' ', ' ', '░', '░', '■', '░', 'S'],
 ['░', '■', ' ', ' ', '░', ' ', '░', ' ', '■', '░'],
 ['■', '░', ' ', ' ', '░', ' ', '■', ' ', ' ', ' '],
 [' ', '░', ' ', '░', ' ', ' ', ' ', ' ', '░', '░'],
 [' ', ' ', '░', ' ', '■', ' ', ' ', ' ', ' ', ' ']]

In [309]:
#we define a function to get possible next movement
#in the form of neighbors that are not wall
#with djikstra, we want the queue to be ordered based on the cost of movement
def getNeighbors(point, grid, current_steps):
    queue = []
    num_rows = len(grid)
    num_cols = len(grid[0])
    x = point.x
    y = point.y
    steps=current_steps+1
    #up
    if point.x > 0:
        newpoint = Point(x=x-1,y=y,value=grid[x-1][y],steps=steps)
        queue.append(newpoint)
    #down
    if x < num_rows -1:
        newpoint = Point(x=x+1,y=y,value=grid[x+1][y],steps=steps)
        queue.append(newpoint)
    #left
    if y > 0:
        newpoint = Point(x=x,y=y-1,value=grid[x][y-1],steps=steps)
        queue.append(newpoint)
    #right
    if y < num_cols -1:
        newpoint = Point(x=x,y=y+1,value=grid[x][y+1],steps=steps)
        queue.append(newpoint)
    queue = [m for m in queue if m.value != '■']
    queue = sorted(queue, key=lambda x: costs[x.repr])
    return queue

In [310]:
#dijkstra

frontier = getNeighbors(starting_point, grid,0)
came_from = {}
cost_so_far = {}

came_from[starting_point.repr] = None
cost_so_far[starting_point.repr] = 0
for f in frontier:
    came_from[f.repr] = starting_point
    cost_so_far[f.repr] = costs[f.repr]

min_cost = 999999
while frontier:
    current = frontier.pop(0)
    cost = cost_so_far[current.repr]
    steps = current.steps
    print(current)
    for n in getNeighbors(current, grid,steps):
        new_cost = cost + costs[n.repr] + steps
        if n.value != 'E' and n.repr not in cost_so_far:
            frontier.append(n)
            came_from[n.repr] = current
            cost_so_far[n.repr] = new_cost
        elif n.value == 'E':
            if new_cost <= min_cost:
                came_from[n.repr] = current
                min_steps = n.steps
            pass


{'x': 6, 'y': 4, 'value': ' '}
{'x': 5, 'y': 3, 'value': ' '}
{'x': 5, 'y': 5, 'value': ' '}
{'x': 4, 'y': 4, 'value': '░'}
{'x': 7, 'y': 4, 'value': '░'}
{'x': 6, 'y': 5, 'value': '░'}
{'x': 4, 'y': 3, 'value': ' '}
{'x': 5, 'y': 2, 'value': ' '}
{'x': 4, 'y': 5, 'value': ' '}
{'x': 8, 'y': 4, 'value': '░'}
{'x': 6, 'y': 6, 'value': ' '}
{'x': 4, 'y': 2, 'value': '░'}
{'x': 6, 'y': 2, 'value': ' '}
{'x': 5, 'y': 1, 'value': '░'}
{'x': 9, 'y': 4, 'value': ' '}
{'x': 8, 'y': 3, 'value': ' '}
{'x': 8, 'y': 5, 'value': '░'}
{'x': 7, 'y': 6, 'value': '░'}
{'x': 6, 'y': 7, 'value': '░'}
{'x': 3, 'y': 2, 'value': ' '}
{'x': 4, 'y': 1, 'value': ' '}
{'x': 7, 'y': 2, 'value': ' '}
{'x': 6, 'y': 1, 'value': ' '}
{'x': 5, 'y': 0, 'value': ' '}
{'x': 9, 'y': 5, 'value': ' '}
{'x': 8, 'y': 2, 'value': '░'}
{'x': 8, 'y': 6, 'value': '░'}
{'x': 7, 'y': 7, 'value': ' '}
{'x': 5, 'y': 7, 'value': '░'}
{'x': 3, 'y': 1, 'value': ' '}
{'x': 4, 'y': 0, 'value': ' '}
{'x': 7, 'y': 1, 'value': '░'}
{'x': 6,

In [312]:
grid

[[' ', ' ', '■', ' ', '░', ' ', '░', ' ', ' ', '░'],
 ['■', ' ', ' ', '■', '░', '■', ' ', '░', '■', '░'],
 ['░', ' ', '■', ' ', ' ', '■', '■', ' ', '░', ' '],
 ['░', ' ', ' ', '■', '■', '■', ' ', '░', ' ', '■'],
 [' ', ' ', '░', ' ', '░', ' ', '■', '■', '░', ' '],
 [' ', '░', ' ', ' ', 'S', ' ', '■', '░', ' ', ' '],
 ['░', ' ', ' ', '■', ' ', '░', ' ', '░', '■', '■'],
 ['░', '░', ' ', '■', '░', '■', '░', ' ', '■', ' '],
 [' ', '░', '░', ' ', '░', '░', '░', '■', '░', '░'],
 [' ', '■', '■', '■', ' ', ' ', '■', 'E', ' ', ' ']]

In [311]:
#draw the path
current = came_from[ending_point.repr]
path = []
while current.repr != starting_point.repr:
    path.append(current.repr)
    current = came_from[current.repr]
for p in path:
    cell = [int(x) for x in list(p)]
    grid[cell[0]][cell[1]] = '*'

KeyError: '97'